In [186]:
from IPython import get_ipython
if get_ipython():
    get_ipython().run_line_magic("load_ext", "autoreload")
    get_ipython().run_line_magic("autoreload", "2")

import numpy as np
import pandas as pd
import torch
import math

import xarray as xr

import matplotlib.pyplot as plt
import seaborn as sns

import collections

import latenta as la
la.logger.setLevel("INFO")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [187]:
cells = la.Dim(pd.Series(range(3), name = "cell").astype(str))
axes = la.Dim(pd.Series(range(2), name = "axis").astype(str))

In [188]:
q = la.distributions.Normal(
    la.Fixed(np.array([[0., 0.], [1., 0.], [1., 0.]]), definition = la.Definition([cells, axes])),
    scale = la.Fixed(0.1),
    transforms = [la.transforms.Circular()]
)

In [189]:
p = la.distributions.Normal(
    la.Parameter(-1., definition = la.Definition([cells, axes])),
    transforms = [la.transforms.Circular()]
)

In [190]:
latent = la.Latent(p, q)

In [191]:
observed = la.posterior.Observed(latent)
observed.sample(20)
elbo_before = observed.elbo

  0%|          | 0/20 [00:00<?, ?it/s]

In [192]:
    latent.reset_recursive()
    inference = la.infer.svi.SVI(
        latent,
        [la.infer.loss.ELBO()],
        la.infer.optim.Adam(lr = 0.05),
        # subsamplers = {"cell":la.infer.subsampling.WeightedSubsampler(weights_value, 1000)},
        device = "cpu"
    )
    trainer = la.infer.trainer.Trainer(inference)
    trace = trainer.train(10000)

  0%|                                                                                                         …

In [199]:
assert ((q.loc.prior() - p.loc.prior())**2).detach().sum().sqrt() < 0.5

In [200]:
observed = la.posterior.Observed(latent)
observed.sample(20)
observed.elbo

assert elbo_before < observed.elbo
assert observed.elbo - elbo_before > 3

  0%|          | 0/20 [00:00<?, ?it/s]